In [87]:
import pandas as pd
import ast
import numpy as np

# --- 1. CARREGAR E PREPARAR O DATAFRAME ---
# 'low_memory=False' é usado para evitar problemas de tipo de dados mistos (DtypeWarning).
print("--- Carregando o Dataset ---")
try:
    df = pd.read_csv('archive/movies_metadata.csv', low_memory=False)
except FileNotFoundError:
    print("ERRO: Arquivo 'archive/movies_metadata.csv' não encontrado. Verifique o caminho.")
    exit()

# Definir todas as colunas JSON que potencialmente existem no arquivo
ALL_JSON_COLS = ['genres', 'belongs_to_collection', 'production_companies', 
                 'production_countries', 'spoken_languages', 'cast', 'crew']

# Lista final das colunas que realmente existem no seu DataFrame
JSON_COLS_TO_PROCESS = [col for col in ALL_JSON_COLS if col in df.columns]
print(f"Colunas JSON que serão processadas: {JSON_COLS_TO_PROCESS}")

# --- 2. FUNÇÃO DE DESSERIALIZAÇÃO SEGURA ---
def safe_literal_eval(val):
    """Converte strings literais (JSON/Python) em objetos Python, tratando nulos e erros."""
    if pd.isna(val) or val in ['', 'None']:
        return None
    try:
        # ast.literal_eval é mais seguro que eval() e trata strings literais de Python
        return ast.literal_eval(val)
    except:
        return None

# --- 3. APLICAR DESSERIALIZAÇÃO (CRIA AS COLUNAS '_obj') ---
print("--- Aplicando Desserialização ---")
for col in JSON_COLS_TO_PROCESS:
    # 3.1 Cria a coluna temporária de objeto (ex: df['genres_obj'])
    df[f'{col}_obj'] = df[col].apply(safe_literal_eval)
    
    # 3.2 Preenchimento de nulos para garantir que o objeto seja Lista ou Dicionário
    # (Para evitar erros nos passos de concatenação/contagem)
    if col == 'belongs_to_collection':
        # Deve ser Dicionário {} para filmes sem coleção
        df[f'{col}_obj'] = df[f'{col}_obj'].apply(lambda x: x if isinstance(x, dict) else {})
    elif isinstance(df[f'{col}_obj'].iloc[0], list) or col not in ['belongs_to_collection']: 
        # Deve ser Lista [] para colunas como genres, companies, etc.
        df[f'{col}_obj'] = df[f'{col}_obj'].apply(lambda x: x if isinstance(x, list) else [])


# --- 4. TRATAMENTO DA COLUNA 'belongs_to_collection' (Dicionário Único) ---

if 'belongs_to_collection' in JSON_COLS_TO_PROCESS:
    # Extrai o nome da coleção e coloca em uma coluna simples
    df['collection_name'] = df['belongs_to_collection_obj'].apply(
        lambda x: x.get('name') if isinstance(x, dict) else None
    )


# --- 5. NORMALIZAÇÃO DE LISTAS (Contagem e Lista Simples de Nomes) ---

# Colunas que são listas e que foram criadas no passo 3 (excluindo o dicionário 'belongs_to_collection')
LIST_COLS_OBJ = [col for col in JSON_COLS_TO_PROCESS if col != 'belongs_to_collection']

print("--- Criando Contagens e Listas Simples ---")

for col in LIST_COLS_OBJ:
    obj_col = f'{col}_obj' 
    
    # 5.1. Extrai o número de itens na lista (ex: genres_count)
    df[f'{col}_count'] = df[obj_col].apply(lambda x: len(x) if isinstance(x, list) else 0)
    
    # 5.2. Cria uma coluna de texto simples (ex: "Action|Comedy")
    def extract_names(lista):
        if isinstance(lista, list):
            # Extrai o nome de cada dicionário na lista (com tratamento seguro)
            names = [d.get('name') for d in lista if isinstance(d, dict) and d.get('name')]
            return '|'.join(names)
        return ''

    df[f'{col}_list'] = df[obj_col].apply(extract_names)


# --- 6. REMOÇÃO FINAL DAS COLUNAS JSON ORIGINAIS E OBJETOS TEMPORÁRIOS ---

# Colunas originais com JSON/Python object:
cols_to_drop = JSON_COLS_TO_PROCESS

# Colunas temporárias (Python objects) criadas na Etapa 3:
cols_to_drop.extend([f'{col}_obj' for col in JSON_COLS_TO_PROCESS])

# Remove as colunas complexas, preservando apenas as colunas limpas (e as contagens/listas)
df = df.drop(columns=cols_to_drop, errors='ignore')


# --- 7. EXIBIR O RESULTADO FINAL ---
print("\n" + "="*50)
print("✅ LIMPEZA E NORMALIZAÇÃO BÁSICA CONCLUÍDAS.")
print("="*50)

print("\nColunas Restantes no DataFrame Limpo:")
print(df.columns.tolist())

print("\nExemplo de Dados Limpos:")
# Exibe as novas colunas limpas: o nome da coleção e as contagens/listas
print(df[['title', 'collection_name', 'genres_count', 'genres_list', 'production_companies_count']].head())

--- Carregando o Dataset ---
Colunas JSON que serão processadas: ['genres', 'belongs_to_collection', 'production_companies', 'production_countries', 'spoken_languages']
--- Aplicando Desserialização ---
--- Criando Contagens e Listas Simples ---

✅ LIMPEZA E NORMALIZAÇÃO BÁSICA CONCLUÍDAS.

Colunas Restantes no DataFrame Limpo:
['adult', 'budget', 'homepage', 'id', 'imdb_id', 'original_language', 'original_title', 'overview', 'popularity', 'poster_path', 'release_date', 'revenue', 'runtime', 'status', 'tagline', 'title', 'video', 'vote_average', 'vote_count', 'collection_name', 'genres_count', 'genres_list', 'production_companies_count', 'production_companies_list', 'production_countries_count', 'production_countries_list', 'spoken_languages_count', 'spoken_languages_list']

Exemplo de Dados Limpos:
                         title                 collection_name  genres_count  \
0                    Toy Story            Toy Story Collection             3   
1                      Jumanj

In [88]:
df_filtrado = df

In [90]:
df_filtrado.drop(columns=['homepage', 'imdb_id', 'budget', 'poster_path', 'collection_name', 'production_companies_count', 'production_companies_list', 'production_countries_count'], inplace=True)

In [91]:
df_filtrado.rename(columns={'adult': 'Adulto', 'original_language': 'Idioma Original', 'original_title': 'Título Original', 'overview': "Descrição", 'popularity': 'Popularidade', 'release_date': 'Data de Lançamento', 'revenue': 'Receita', 'runtime': 'Duração', 'status': 'Status', 'production_countries_list': 'País de Origem', 'spoken_languages_count': 'Idiomas falados no filme', 'spoken_languages_list': 'Lista de Idiomas', 'title': 'Título em inglês'}, inplace=True)

In [92]:
df_brazil = df_filtrado[df_filtrado['País de Origem'] == 'Brazil']

In [93]:
df_brazil.sort_values('Data de Lançamento',ascending=False)

,Adulto,id,Idioma Original,Título Original,Descrição,Popularidade,Data de Lançamento,Receita,Duração,Status,tagline,Título em inglês,video,vote_average,vote_count,genres_count,genres_list,País de Origem,Idiomas falados no filme,Lista de Idiomas
44591,False,448763,pt,Amor.com,It's a love story between a fashion blogger an...,1.957136,2017-06-01,0.0,92.0,Released,NaN,Amor.com,False,6.8,17.0,1,Romance,Brazil,1,Português
42827,False,430128,pt,Internet - O Filme,"In a convention of youtubers, the characters e...",2.75527,2017-02-23,0.0,0.0,Released,NaN,Internet - O Filme,False,4.2,52.0,1,Comedy,Brazil,1,Português
42149,False,428645,pt,Eu Fico Loko,NaN,1.948382,2017-01-12,0.0,NaN,Released,NaN,Eu Fico Loko,False,8.3,22.0,1,Comedy,Brazil,1,Português
42157,False,227932,pt,Minha Mãe é Uma Peça 2,"Dona Hermínia is back, but now rich and famous...",4.117801,2016-12-22,0.0,96.0,Released,NaN,My Mom Is a Character 2,False,7.8,100.0,1,Comedy,Brazil,1,Português
41720,False,296288,pt,Tamo Junto,"Guy ends his relationship, re-encounter his be...",0.227697,2016-12-08,0.0,100.0,Released,NaN,Tamo Junto,False,3.5,2.0,1,Comedy,Brazil,1,Português
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16757,False,59990,pt,O Pagador de Promessas,Zé is a very poor man from the Brazilian count...,0.904016,1962-04-17,0.0,98.0,Released,The story of a vow that a woman broke and a ma...,The Given Word,False,7.2,9.0,1,Drama,Brazil,1,Português
36814,False,28525,pt,Os Cafajestes,This film captures the criminal behavior of tw...,0.107987,1962-03-24,0.0,100.0,Released,NaN,The Unscrupulous Ones,False,7.0,3.0,1,Drama,Brazil,1,Português
38027,False,146904,pt,"Rio, Zona Norte",Setting up the gracefully jarring dichotomies ...,0.072383,1957-08-26,0.0,90.0,Released,NaN,"Rio, Zona Norte",False,5.0,1.0,0,,Brazil,1,Português
38028,False,146075,pt,"Rio, 40 graus",Banned by Brazil’s Federal Department of Publi...,0.192623,1955-08-24,0.0,100.0,Released,NaN,Rio 100 Degrees F.,False,3.0,2.0,1,Drama,Brazil,1,Português


In [94]:
df_filtrado = df_filtrado[['id', 'Título Original', 'Título em inglês', 'País de Origem', 'Idioma Original', 'Idiomas falados no filme', 'Lista de Idiomas', 'genres_list', 'vote_count', 'vote_average', 'Popularidade', 'Duração', 'Data de Lançamento', 'Adulto', 'Status']]

In [95]:
df_analise = df_filtrado[['Título Original', 'Título em inglês', 'País de Origem', 'Idioma Original']].copy() 

# 2. Cria a coluna booleana 'Analise'
# Verifica se os valores são iguais e armazena o resultado (True/False)
df_analise['Analise'] = df_filtrado['Título Original'] == df_filtrado['Título em inglês']

df_analise.loc[df_analise['Analise'] == False]

,Título Original,Título em inglês,País de Origem,Idioma Original,Analise
28,La Cité des Enfants Perdus,The City of Lost Children,France|Germany|Spain,fr,False
29,摇啊摇，摇到外婆桥,Shanghai Triad,China|France,zh,False
32,"Guillaumet, les ailes du courage",Wings of Courage,France|United States of America,fr,False
57,Il postino,The Postman,Belgium|France|Italy,it,False
58,Le confessionnal,The Confessional,Canada,fr,False
...,...,...,...,...,...
45453,Maa,Mom,India,hi,False
45455,San Michele aveva un gallo,St. Michael Had a Rooster,,it,False
45461,رگ خواب,Subdue,Iran,fa,False
45462,Siglo ng Pagluluwal,Century of Birthing,Philippines,tl,False


In [96]:
df_analise

,Título Original,Título em inglês,País de Origem,Idioma Original,Analise
0,Toy Story,Toy Story,United States of America,en,True
1,Jumanji,Jumanji,United States of America,en,True
2,Grumpier Old Men,Grumpier Old Men,United States of America,en,True
3,Waiting to Exhale,Waiting to Exhale,United States of America,en,True
4,Father of the Bride Part II,Father of the Bride Part II,United States of America,en,True
...,...,...,...,...,...
45461,رگ خواب,Subdue,Iran,fa,False
45462,Siglo ng Pagluluwal,Century of Birthing,Philippines,tl,False
45463,Betrayal,Betrayal,United States of America,en,True
45464,Satana likuyushchiy,Satan Triumphant,Russia,en,False


Analisando o DataSet Keywords

In [97]:
import pandas as pd
import ast
import numpy as np

# --- 1. CARREGAR O DATAFRAME ---
# Assumindo que o arquivo keywords.csv está na mesma pasta raiz
try:
    df_keywords = pd.read_csv('archive/keywords.csv')
except FileNotFoundError:
    print("ERRO: Arquivo 'keywords.csv' não encontrado. Verifique o caminho.")
    exit()

# --- 2. FUNÇÃO DE DESSERIALIZAÇÃO SEGURA ---
def safe_literal_eval(val):
    """Converte strings literais (como JSON) em objetos Python, tratando nulos e erros."""
    if pd.isna(val) or val in ['', 'None', '[]']:
        return []
    try:
        return ast.literal_eval(val)
    except:
        return []

# --- 3. APLICAR DESSERIALIZAÇÃO ---
df_keywords['keywords_obj'] = df_keywords['keywords'].apply(safe_literal_eval)

# --- 4. NORMALIZAÇÃO: Achatando a Lista de Dicionários ---

# Achata a coluna 'keywords_obj', criando uma linha para cada palavra-chave por filme.
keywords_normalized = pd.json_normalize(
    df_keywords.to_dict('records'),  # Converte o DF para o formato que json_normalize espera
    record_path='keywords_obj',      # Onde está a lista que queremos achatar
    meta=['id'],                     # Mantém a coluna 'id' do filme como metadado
    record_prefix='keyword_'         # Prefixo para as colunas extraídas (ex: keyword_id, keyword_name)
)

# --- 5. LIMPEZA FINAL ---

# Remove a coluna original e a coluna temporária de objetos
keywords_normalized = keywords_normalized.drop(columns=['keywords_obj'], errors='ignore')


# --- 6. EXIBIR O RESULTADO ---
print("\n" + "="*50)
print("✅ NORMALIZAÇÃO DE KEYWORDS CONCLUÍDA.")
print("="*50)

print("\nExemplo de Dados Normalizados (Palavra-Chave por Linha):")
# Exibe as colunas: ID do filme, ID da keyword e Nome da keyword
print(keywords_normalized[['id', 'keyword_id', 'keyword_name']].head(10))


✅ NORMALIZAÇÃO DE KEYWORDS CONCLUÍDA.

Exemplo de Dados Normalizados (Palavra-Chave por Linha):
     id  keyword_id       keyword_name
0   862         931           jealousy
1   862        4290                toy
2   862        5202                boy
3   862        6054         friendship
4   862        9713            friends
5   862        9823            rivalry
6   862      165503      boy next door
7   862      170722            new toy
8   862      187065  toy comes to life
9  8844       10090         board game


In [98]:
keywords_normalized

,keyword_id,keyword_name,id
0,931,jealousy,862
1,4290,toy,862
2,5202,boy,862
3,6054,friendship,862
4,9713,friends,862
...,...,...,...
158675,11800,mockumentary,289923
158676,10703,tragic love,439050
158677,2679,artist,111109
158678,14531,play,111109


In [99]:
df_filtrado.head(5)

,id,Título Original,Título em inglês,País de Origem,Idioma Original,Idiomas falados no filme,Lista de Idiomas,genres_list,vote_count,vote_average,Popularidade,Duração,Data de Lançamento,Adulto,Status
0,862,Toy Story,Toy Story,United States of America,en,1,English,Animation|Comedy|Family,5415.0,7.7,21.946943,81.0,1995-10-30,False,Released
1,8844,Jumanji,Jumanji,United States of America,en,2,English|Français,Adventure|Fantasy|Family,2413.0,6.9,17.015539,104.0,1995-12-15,False,Released
2,15602,Grumpier Old Men,Grumpier Old Men,United States of America,en,1,English,Romance|Comedy,92.0,6.5,11.7129,101.0,1995-12-22,False,Released
3,31357,Waiting to Exhale,Waiting to Exhale,United States of America,en,1,English,Comedy|Drama|Romance,34.0,6.1,3.859495,127.0,1995-12-22,False,Released
4,11862,Father of the Bride Part II,Father of the Bride Part II,United States of America,en,1,English,Comedy,173.0,5.7,8.387519,106.0,1995-02-10,False,Released


In [100]:
keywords_normalized.head(5)

,keyword_id,keyword_name,id
0,931,jealousy,862
1,4290,toy,862
2,5202,boy,862
3,6054,friendship,862
4,9713,friends,862


In [101]:
keywords_normalized.groupby(by= ['id'])

In [102]:
import pandas as pd

# 1. Agrupar as palavras-chave por ID de filme
# Usamos o ID do filme ('id') para agrupar, e a função .agg() para juntar
# todos os nomes das palavras-chave ('keyword_name') em uma única string,
# separada por um pipe '|'.

keywords_agregadas = keywords_normalized.groupby('id')['keyword_name'].agg(lambda x: '|'.join(x)).reset_index()

# 2. Renomear a coluna agregada
# A coluna resultante da agregação é nomeada 'keyword_name', renomeamos para ser descritiva
keywords_agregadas.rename(columns={'keyword_name': 'keywords_list'}, inplace=True)

# 3. Juntar (Merge) ao DataFrame principal (df_filtrado)
# O merge é feito usando a coluna 'id', que é comum a ambos os DataFrames.

# A coluna de ID no df_filtrado pode ter sido convertida para string ou float. 
# Para evitar problemas, garantimos que ambas as colunas 'id' sejam tratadas como números inteiros,
# caso ainda não estejam (o que é comum neste dataset de filmes).
try:
    df_filtrado['id'] = pd.to_numeric(df_filtrado['id'], errors='coerce').astype('Int64')
    keywords_agregadas['id'] = pd.to_numeric(keywords_agregadas['id'], errors='coerce').astype('Int64')
except:
    print("Aviso: Falha na conversão de ID para inteiro, usando tipo existente.")


df_filtrado = pd.merge(
    df_filtrado, 
    keywords_agregadas, 
    on='id', 
    how='left' # Usamos 'left' para manter todos os filmes em df_filtrado
)

# 4. Visualizar o Resultado
print("Agregação de Keywords Concluída. Novas colunas:")
print(df_filtrado[['Título Original', 'keywords_list']].head())

Agregação de Keywords Concluída. Novas colunas:
               Título Original  \
0                    Toy Story   
1                      Jumanji   
2             Grumpier Old Men   
3            Waiting to Exhale   
4  Father of the Bride Part II   

                                       keywords_list  
0  jealousy|toy|boy|friendship|friends|rivalry|bo...  
1  board game|disappearance|based on children's b...  
2   fishing|best friend|duringcreditsstinger|old men  
3  based on novel|interracial relationship|single...  
4  baby|midlife crisis|confidence|aging|daughter|...  


C:\Users\kauan\AppData\Local\Temp\ipykernel_36088\244762412.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado['id'] = pd.to_numeric(df_filtrado['id'], errors='coerce').astype('Int64')


In [103]:
df_filtrado.iloc[0]

id                                                                        862
Título Original                                                     Toy Story
Título em inglês                                                    Toy Story
País de Origem                                       United States of America
Idioma Original                                                            en
Idiomas falados no filme                                                    1
Lista de Idiomas                                                      English
genres_list                                           Animation|Comedy|Family
vote_count                                                             5415.0
vote_average                                                              7.7
Popularidade                                                        21.946943
Duração                                                                  81.0
Data de Lançamento                                              

In [104]:
df_filtrado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 16 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   id                        45463 non-null  Int64  
 1   Título Original           45466 non-null  object 
 2   Título em inglês          45460 non-null  object 
 3   País de Origem            45466 non-null  object 
 4   Idioma Original           45455 non-null  object 
 5   Idiomas falados no filme  45466 non-null  int64  
 6   Lista de Idiomas          45466 non-null  object 
 7   genres_list               45466 non-null  object 
 8   vote_count                45460 non-null  float64
 9   vote_average              45460 non-null  float64
 10  Popularidade              45461 non-null  object 
 11  Duração                   45203 non-null  float64
 12  Data de Lançamento        45379 non-null  object 
 13  Adulto                    45466 non-null  object 
 14  Status

In [105]:
df_filtrado.sort_values('Data de Lançamento')

,id,Título Original,Título em inglês,País de Origem,Idioma Original,Idiomas falados no filme,Lista de Idiomas,genres_list,vote_count,vote_average,Popularidade,Duração,Data de Lançamento,Adulto,Status,keywords_list
19730,<NA>,"[{'iso_639_1': 'en', 'name': 'English'}]",NaN,,104.0,0,,Carousel Productions|Vision View Entertainment...,NaN,NaN,NaN,NaN,1,- Written by Ørnås,NaN,NaN
29503,<NA>,"[{'iso_639_1': 'ja', 'name': '日本語'}]",NaN,,68.0,0,,Aniplex|GoHands|BROSTA TV|Mardock Scramble Pro...,NaN,NaN,NaN,NaN,12,Rune Balot goes to a casino connected to the ...,NaN,NaN
34940,315946,Passage de Venus,Passage of Venus,France,xx,1,No Language,Documentary,19.0,6.0,0.480371,1.0,1874-12-09,False,Released,silent film|science|astronomy|venus the planet...
34937,194079,Sallie Gardner at a Gallop,Sallie Gardner at a Gallop,United States of America,en,1,No Language,Documentary,25.0,6.2,0.327841,1.0,1878-06-14,False,Released,horse|stop motion|animation|black and white|si...
41602,426903,Buffalo Running,Buffalo Running,United States of America,en,1,No Language,Documentary,7.0,5.4,0.229221,1.0,1883-11-19,False,Released,running|buffalo|photography|black and white|short
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45148,438910,Konstruktor krasnogo tsveta -1993,Engineering Red,,ru,0,,,2.0,6.0,0.001586,76.0,NaN,False,Released,NaN
45203,433711,All Superheroes Must Die 2: The Last Superhero,All Superheroes Must Die 2: The Last Superhero,,en,1,English,Mystery|Science Fiction,1.0,4.0,0.00022,74.0,NaN,False,Released,NaN
45338,335251,The Land Where the Blues Began,The Land Where the Blues Began,,en,0,,,0.0,0.0,0.0,0.0,NaN,False,Released,NaN
45410,449131,Aprel,Aprel,Russia,ru,0,,Drama|Crime,1.0,6.0,0.008903,NaN,NaN,False,Released,NaN


In [106]:
df_filtrado.dropna(inplace= True)

In [107]:
df_filtrado.loc[(df_filtrado['País de Origem'] == "") & (df_filtrado['Idioma Original'] == "")]

,id,Título Original,Título em inglês,País de Origem,Idioma Original,Idiomas falados no filme,Lista de Idiomas,genres_list,vote_count,vote_average,Popularidade,Duração,Data de Lançamento,Adulto,Status,keywords_list


In [108]:
df_filtrado.loc[(df_filtrado['País de Origem'] == "")].groupby('Idioma Original').count()

,id,Título Original,Título em inglês,País de Origem,Idiomas falados no filme,Lista de Idiomas,genres_list,vote_count,vote_average,Popularidade,Duração,Data de Lançamento,Adulto,Status,keywords_list
Idioma Original,,,,,,,,,,,,,,,
cn,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3
cs,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
cy,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
da,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7
de,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24
el,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4
en,1836,1836,1836,1836,1836,1836,1836,1836,1836,1836,1836,1836,1836,1836,1836
es,22,22,22,22,22,22,22,22,22,22,22,22,22,22,22
et,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


In [109]:
contagem_por_idioma = df_filtrado.loc[df_filtrado['País de Origem'] == ""].groupby('Idioma Original').size()

print("Contagem de Filmes (sem País de Origem) por Idioma:")
print(contagem_por_idioma)

Contagem de Filmes (sem País de Origem) por Idioma:
Idioma Original
cn       3
cs       1
cy       1
da       7
de      24
el       4
en    1836
es      22
et       1
eu       1
fa       2
fi       9
fr      31
he       1
hi      17
hu       2
is       2
it      34
ja      12
ka       1
kn       1
ko       3
lv       1
ml       1
mr       1
nb       1
nl      11
no       2
pl       3
pt       7
ru      13
sv       8
te       1
tr       2
uz       1
vi       1
xx       1
zh       7
dtype: int64


In [110]:
import pandas as pd

# 🗺️ Dicionário de Mapeamento Idioma -> País (Simplificado para o Exemplo)
country_map = {
    'en': 'USA',         
    'it': 'Italy',      
    'fr': 'France',     
    'de': 'Germany',   
    'es': 'Spain',     
    'hi': 'India',      
    'ru': 'Russia',    
    'fi': 'Finland',   
    'tr': 'Turkey',    
    'nl': 'Netherlands',
    'ja': 'Japan',      
    'sv': 'Sweden',     
    'pt': 'Brazil',     
    'ko': 'South Korea',
    'zh': 'China',      
    'cn': 'China',      
    'el': 'Greece',     
    'pl': 'Poland',     
    'da': 'Denmark',    
    'ar': 'Egypt',      
    'fa': 'Iran',       
    'hu': 'Hungary',    
    'no': 'Norway',     
    'cs': 'Czechia',    
    'te': 'India',      
    'uk': 'Ukraine',    
    'he': 'Israel',     
    'vi': 'Vietnam',    
    'is': 'Iceland',    
    'et': 'Estonia',    
    'cy': 'UK',         
    'sq': 'Albania',    
    'ml': 'India',      
    'mr': 'India',      
    'bn': 'Bangladesh', 
    'ur': 'Pakistan',   
    'uz': 'Uzbekistan', 
    'ab': 'Georgia',    
    'ka': 'Georgia',    
    'eu': 'Spain',      
    'kn': 'India',      
    'ta': 'India',      
    'fy': 'Netherlands',
    'nb': 'Norway',     
    'xx': 'N/A'         
}

# --- 1. Criar o filtro booleano para as linhas sem País de Origem ---
filtro_sem_pais = df_filtrado['País de Origem'] == ""

# --- 2. Aplicar o mapeamento (map) somente às linhas filtradas ---

# 2.1. Seleciona a coluna 'Idioma Original' SOMENTE para as linhas filtradas.
# 2.2. Aplica o dicionário country_map a esses valores de idioma.
# 2.3. O resultado é a nova Série de países (ex: 'USA', 'France', 'Brazil').
novos_paises = df_filtrado.loc[filtro_sem_pais, 'Idioma Original'].map(country_map)

# --- 3. Atribuir os novos valores de volta à coluna 'País de Origem' ---

# Usamos .loc novamente para ATRIBUIR os novos valores APENAS às linhas filtradas
df_filtrado.loc[filtro_sem_pais, 'País de Origem'] = novos_paises

# 4. Visualização de uma amostra para confirmar a imputação
print("✅ Imputação de País de Origem com base no Idioma concluída.")
print("\nExemplo de Filmes Onde o País foi Preenchido:")
# Filtra novamente as linhas que estavam vazias e que agora foram preenchidas
# (Note que algumas podem continuar vazias se o idioma original não estava no dicionário)
print(df_filtrado.loc[df_filtrado['País de Origem'] != ""].head(10))

✅ Imputação de País de Origem com base no Idioma concluída.

Exemplo de Filmes Onde o País foi Preenchido:
       id              Título Original             Título em inglês  \
0     862                    Toy Story                    Toy Story   
1    8844                      Jumanji                      Jumanji   
2   15602             Grumpier Old Men             Grumpier Old Men   
3   31357            Waiting to Exhale            Waiting to Exhale   
4   11862  Father of the Bride Part II  Father of the Bride Part II   
5     949                         Heat                         Heat   
6   11860                      Sabrina                      Sabrina   
8    9091                 Sudden Death                 Sudden Death   
9     710                    GoldenEye                    GoldenEye   
10   9087       The American President       The American President   

                             País de Origem Idioma Original  \
0                  United States of America     

In [111]:
df_filtrado.sort_values('Duração')

,id,Título Original,Título em inglês,País de Origem,Idioma Original,Idiomas falados no filme,Lista de Idiomas,genres_list,vote_count,vote_average,Popularidade,Duração,Data de Lançamento,Adulto,Status,keywords_list
35594,153717,Aschenputtel,Aschenputtel,Austria|Germany,de,1,Deutsch,Fantasy|Family,4.0,8.3,0.781805,0.0,2010-12-24,False,Released,fairy tale|woman director|fairy tale|woman dir...
35739,284050,Mr. Kaplan,Mr. Kaplan,Spain|Uruguay|Germany,es,1,Español,Drama|Comedy,5.0,5.6,0.968647,0.0,2014-11-21,False,Released,suspense|suspense
35793,266249,Das Lied einer Nacht,Das Lied einer Nacht,Germany,de,0,,Comedy,1.0,7.0,0.323312,0.0,1932-11-06,False,Released,musical|musical
45150,67431,Gas-oil,Hi-Jack Highway,France,fr,1,Français,Drama,0.0,0.0,0.383766,0.0,1955-10-18,False,Released,truck driver|french noir
45180,298787,3 बजे,3 A.M,India,hi,1,हिन्दी,Thriller|Horror,3.0,2.7,0.353854,0.0,2014-09-26,False,Released,hindi|horror movie
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19158,208988,The War,The War,United States of America,en,1,English,Documentary|History|War,3.0,5.3,0.487489,874.0,2007-09-23,False,Released,post world war ii
27855,125336,The Story of Film: An Odyssey,The Story of Film: An Odyssey,United Kingdom,en,1,English,Documentary,26.0,8.8,1.50251,900.0,2011-09-03,False,Released,cinema|nouvelle vague|hindi cinema|cinema novo...
13953,67463,Heimat: Eine deutsche Chronik,Heimat: A Chronicle of Germany,Germany,de,0,,Drama|History,1.0,9.0,0.016204,925.0,1984-09-16,False,Released,germany|epic|longest film
40938,126820,Baseball,Baseball,USA,en,1,English,Documentary,4.0,8.5,0.145073,1140.0,1994-09-18,False,Released,history


In [112]:
df_filtrado

,id,Título Original,Título em inglês,País de Origem,Idioma Original,Idiomas falados no filme,Lista de Idiomas,genres_list,vote_count,vote_average,Popularidade,Duração,Data de Lançamento,Adulto,Status,keywords_list
0,862,Toy Story,Toy Story,United States of America,en,1,English,Animation|Comedy|Family,5415.0,7.7,21.946943,81.0,1995-10-30,False,Released,jealousy|toy|boy|friendship|friends|rivalry|bo...
1,8844,Jumanji,Jumanji,United States of America,en,2,English|Français,Adventure|Fantasy|Family,2413.0,6.9,17.015539,104.0,1995-12-15,False,Released,board game|disappearance|based on children's b...
2,15602,Grumpier Old Men,Grumpier Old Men,United States of America,en,1,English,Romance|Comedy,92.0,6.5,11.7129,101.0,1995-12-22,False,Released,fishing|best friend|duringcreditsstinger|old men
3,31357,Waiting to Exhale,Waiting to Exhale,United States of America,en,1,English,Comedy|Drama|Romance,34.0,6.1,3.859495,127.0,1995-12-22,False,Released,based on novel|interracial relationship|single...
4,11862,Father of the Bride Part II,Father of the Bride Part II,United States of America,en,1,English,Comedy,173.0,5.7,8.387519,106.0,1995-02-10,False,Released,baby|midlife crisis|confidence|aging|daughter|...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45453,404604,Maa,Mom,India,hi,1,हिन्दी,Crime|Drama|Thriller,14.0,6.6,1.559596,146.0,2017-07-07,False,Released,physical abuse|sexual assault
45456,84419,House of Horrors,House of Horrors,United States of America,en,1,English,Horror|Mystery|Thriller,8.0,6.3,0.222814,65.0,1946-03-29,False,Released,revenge|murder|serial killer|new york city|scu...
45457,390959,Shadow of the Blair Witch,Shadow of the Blair Witch,USA,en,1,English,Mystery|Horror,2.0,7.0,0.076061,45.0,2000-10-22,False,Released,blair witch
45458,289923,The Burkittsville 7,The Burkittsville 7,United States of America,en,1,English,Horror,1.0,7.0,0.38645,30.0,2000-10-03,False,Released,witch|mythology|legend|serial killer|mockumentary


In [113]:
df_filtrado.to_csv('df_consolidado.csv')

In [114]:
def avaliar_duracao(duracao):
    if duracao > 180:
        conclusao = 'Very Long'
    elif 120 <= duracao <= 180:
         conclusao = 'Long'
    elif 60 <= duracao < 120:
         conclusao = 'Medium'
    elif 30 <= duracao < 60:
         conclusao = 'Short'
    elif duracao < 30:
         conclusao = 'Very Short'

    return conclusao

In [115]:
df_filtrado['Categoria de Tempo'] = df_filtrado['Duração'].apply(avaliar_duracao)

In [116]:
df_filtrado.to_csv('df_consolidado.csv')